In [49]:
# ## Notebook to pair down the game scraping functions to pickup games the were missed.

# games with not advnaced metrics pages threw errors that preventing any info from the game being saved.

# goin to adapt the larger scraping book to 

In [50]:
## Dependencies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import logging
from tqdm import tqdm
import re

from sqlalchemy import create_engine


current_year_url = 'https://www.collegehockeynews.com/schedules/?season=20212022'

In [51]:
## Functions
### Parse the current season schedule / results page

def parse_current_season(url):
        # Initialize variables
    current_date = None
    current_conference = None
    game_notes = None

    # Initialize an empty list to hold the data
    data = []

    # Parse the page with BeautifulSoup
    # Get the page with requests
    response = requests.get(url)

    # Create a BeautifulSoup object
    soup = BeautifulSoup(response.text, 'html.parser')

    # select the table or tables
    tables = soup.find_all('table')

    rows = soup.find_all('tr')

    # Loop through each row to find relevant information
    for row in rows:
        # Check for date row
        if row.get('class') == ['stats-section']:
            current_date = row.find('td').text.strip()
        # Check for conference row
        elif row.get('class') == ['sked-header']:
            current_conference = row.find('td').text.strip()
        # Check for game notes
        elif len(row.find_all('td')) == 2:
            game_notes = row.find_all('td')[1].text.strip()
        # Process rows with game data
        elif row.get('valign') == 'top':
            cells = row.find_all('td')
            if len(cells) >= 9:
                home_team = cells[0].text.strip()
                home_team_link = cells[0].find('a')['href'] if cells[0].find('a') else None
                home_score = cells[1].text.strip()
                away_team = cells[3].text.strip()
                away_team_link = cells[3].find('a')['href'] if cells[3].find('a') else None
                away_score = cells[4].text.strip()
                ot = cells[5].text.strip()
                box_link = cells[7].find('a')['href'] if cells[7].find('a') else None
                metrics_link = cells[8].find('a')['href'] if cells[8].find('a') else None
                # Capture Game Notes
                game_notes_cell = cells[-1].find('small')
                game_notes = game_notes_cell.text.strip() if game_notes_cell else None

                # Append data to the list
                data.append([current_date, current_conference, game_notes, home_team, home_team_link, home_score, away_team, away_team_link, away_score, ot, box_link, metrics_link])
                game_notes = None  # Reset game notes for the next row
    return data

## call the function
data = parse_current_season(current_year_url)


# Create a dataframe from the list

columns = ['Date', 'Conference', 'Game_Notes', 'Home_Team', 'Home_Team_Link', 'Home_Score', 'Away_Team', 'Away_Team_Link', 'Away_Score', 'OT', 'Box_Link', 'Metrics_Link']
df = pd.DataFrame(data, columns=columns)
            
## Extract the day of the week from the date and save in new column
df['Day'] = pd.to_datetime(df['Date']).dt.day_name()
# remove day of the week from date
# format data column as YYYY-MM-DD
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

### Create a new column for the game ID
## Game ID will be a combination of the date and abbreviated team names

# Function to abbreviate the team names
for row in df.itertuples():
    home_team = row.Home_Team
    away_team = row.Away_Team
    home_team_abbr = home_team.split(' ')[-1]
    away_team_abbr = away_team.split(' ')[-1]
    game_id = f'{row.Date}_{home_team_abbr}_{away_team_abbr}'
    df.loc[row.Index, 'Game_ID'] = game_id

# Create a new column for the game ID
df['Game_ID'] = df['Game_ID'].str.replace(',', '')

# Apply the function to the DataFrame
df['Game_ID'] = df.apply(lambda row: f'{row.Date}_{row.Home_Team}_{row.Away_Team}', axis=1)

## Filter out games that have not been played yet
df = df[df['Home_Score'] != '']

# Replace Nan values in metrics column with empty string
# df['Metrics_Link'] = df['Metrics_Link'].fillna('')

print(len(df))
df.head()


1106


C:\Users\Justin\AppData\Local\Temp\ipykernel_18172\3560161213.py:83: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2021-10-02_Army_Providence' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[row.Index, 'Game_ID'] = game_id


,Date,Conference,Game_Notes,Home_Team,Home_Team_Link,Home_Score,Away_Team,Away_Team_Link,Away_Score,OT,Box_Link,Metrics_Link,Day,Game_ID
0,2021-10-02,Non-Conference,,Army,/reports/team/Army/6,0,Providence,/reports/team/Providence/46,7,,/box/final/20211002/arm/prv/,/box/metrics.php?gd=91062,Saturday,2021-10-02_Army_Providence
1,2021-10-02,Non-Conference,,Sacred Heart,/reports/team/Sacred-Heart/51,3,Connecticut,/reports/team/Connecticut/17,6,,/box/final/20211002/sac/con/,/box/metrics.php?gd=91058,Saturday,2021-10-02_Sacred Heart_Connecticut
2,2021-10-02,Non-Conference,,Miami,/reports/team/Miami/30,7,Ferris State,/reports/team/Ferris-State/21,4,,/box/final/20211002/mia/fsu/,/box/metrics.php?gd=91352,Saturday,2021-10-02_Miami_Ferris State
3,2021-10-02,Non-Conference,,Bentley,/reports/team/Bentley/8,0,Northeastern,/reports/team/Northeastern/41,4,,/box/final/20211002/ben/noe/,/box/metrics.php?gd=91061,Saturday,2021-10-02_Bentley_Northeastern
4,2021-10-02,Non-Conference,,St. Thomas,/reports/team/St-Thomas/63,2,St. Cloud State,/reports/team/St-Cloud-State/52,12,,/box/final/20211002/stt/stc/,/box/metrics.php?gd=91445,Saturday,2021-10-02_St. Thomas_St. Cloud State


In [52]:
df.info()

## value count of the conference column
df['Conference'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1106 entries, 0 to 1105
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date            1106 non-null   object
 1   Conference      1106 non-null   object
 2   Game_Notes      1106 non-null   object
 3   Home_Team       1106 non-null   object
 4   Home_Team_Link  1095 non-null   object
 5   Home_Score      1106 non-null   object
 6   Away_Team       1106 non-null   object
 7   Away_Team_Link  1103 non-null   object
 8   Away_Score      1106 non-null   object
 9   OT              1106 non-null   object
 10  Box_Link        1106 non-null   object
 11  Metrics_Link    1082 non-null   object
 12  Day             1106 non-null   object
 13  Game_ID         1106 non-null   object
dtypes: object(14)
memory usage: 121.1+ KB


Conference
Non-Conference                                                                      280
Hockey East                                                                         132
ECAC                                                                                132
Atlantic Hockey                                                                     130
CCHA                                                                                104
NCHC                                                                                 95
Big Ten                                                                              84
ECAC Tournament                                                                      20
Exhibition                                                                           19
AHA Tournament                                                                       13
CCHA Tournament                                                                      13
Big Ten Tournament   

In [53]:
# Drop rows from 'Exhibition' games
df = df[df['Conference'] != 'Exhibition']

## Only keep games with null values in the metrics column
# df = df[df['Metrics_Link'].isnull()]

print(len(df))

1087


In [54]:
## Value count of the conference column
df['Conference'].value_counts()

# value count of metrics column
df['Metrics_Link'].info()

<class 'pandas.core.series.Series'>
Index: 1087 entries, 0 to 1105
Series name: Metrics_Link
Non-Null Count  Dtype 
--------------  ----- 
1082 non-null   object
dtypes: object(1)
memory usage: 17.0+ KB
